In [26]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
   text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [27]:
# tokenize the chars
string_to_int = { ch: i for i, ch in enumerate(chars)} # create a dictionary of the characters
int_to_string = {i:ch for i, ch in enumerate(chars)} # create a reverse dictionary of the characters
encode = lambda words: [string_to_int[char] for char in words]
decode = lambda chars: ''.join([int_to_string[char] for char in chars])

encode_hello = encode('hello')
print(encode_hello)
decoded_hello = decode(encode_hello)
print(decoded_hello)


[61, 58, 65, 65, 68]
hello


In [28]:
import torch

### Using batches to accelerate training process


In [29]:
# check if cuda is available in our hardware
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8 # length of the sequence
batch_size = 4 # number of blocks we are processing in parallel
max_iters = 10000 # number of iterations
learning_rate = 3e-4 # learning rate equivalent to 0.0003
eval_iters = 250


cpu


In [30]:
data = torch.tensor(encode(text), dtype=torch.long) # convert the text to a tensor
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Validation and Training Split


In [31]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print("inputs:")
# print(x.shape)
print(x)
print("target:")
print(y)

inputs:
tensor([[54, 72,  1, 61, 58,  1, 73, 71],
        [62, 72,  1, 75, 68, 62, 56, 58],
        [71,  5, 72, 73, 54, 67, 56, 58],
        [74, 57,  1, 30, 54, 62, 71, 62]])
target:
tensor([[72,  1, 61, 58,  1, 73, 71, 68],
        [72,  1, 75, 68, 62, 56, 58, 11],
        [ 5, 72, 73, 54, 67, 56, 58, 72],
        [57,  1, 30, 54, 62, 71, 62, 58]])


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y  = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
import torch.nn as nn
from torch.nn import functional as F 

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets = None):
        logits = self.token_embeddings_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the functionos
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # becomes (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # becomes (B,1)
            # append sampled index to the running sequence
            index = torch.cat([index, index_next], dim=1) # becomes (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


5RjQ(cJD:c)!G
LdCWV YuS4;KQlk:JLo_cEZR3R1'.uQ0DdsYLMXEq;Hk5oyd7PAUy:8v
5Sk69&,';Evt4n!qAEA83KbTrIpF[,,N?&]kILdC_AQIrtZ9zV t!jC:VtL1wElm8
)k0'g&u9Hse&c,7m0;Vt4iKMS2hR16xMuMnj2
qfF0L﻿Xvrg]AgKzBl_5 3R﻿)!Q;FuO6.G﻿4ilQ0AMq8aDQXB'
e*Lc:ezx(yiZT_A;l.C!bV3;GF[FrFxNnrWb
E mK6]C﻿Zq3pQV8G7c(Z4CSvi5f.y
_4_Ag"3K!7oX
2,_;A)NP5vdp8buG-eAO:YtlbQv
Eg2t;RaUZ-g&3m9Cec:PTL uPm﻿ZR
tM'?-LPnivwi6d*k!BZOS0'TdCP8gx(M*;G*9bX5_K69!3S-69XlttMh'ZOs'je!qAmc6,-:"wa[FXHk&XQ;urN*kz
6g]CUG7owiQzOCsVZWJvxDT?B,OS﻿ob"W;i]'!qAN*4J;L


In [42]:
# create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        print(f'losses at iteration {iter}: {loss.item()}')
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the  loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


2.1833839416503906


***Need to familiarize ***
1. Mean Squared Error(MSE)
2. Gradient Descent (GD)
3. Momentum
4. RMSprop
5. Adam
6. AdamW

In [43]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




"Neind as am wanove ry held are p Bers  toand-smalsind thoeyizang n whe redre hed ce bope."w ar wom t OWong
f 2arizan'lu's
th b at?"

a acit Wen lin can t sthepld t thithil

g Cke warig stw cthe?"

[d theran wacrealy sevirothqus, t siris athacoif
Dor wor there I trg thitth sthe  s; uthimy nd
ish "Ye pie is,"
ADu S; hemubulewng t wed
f 's the. ZOzed hthab. h
"l L1Qha ch lew tlld gro thadlarare sok handenghanjussth.
" aveand ancqurin m alir ritair
loylain ain t atru ybereshand futong.
"
om erone


In [35]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
   context = x[:t+1]
   target = y[t]
   print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
